# PIMA ENSEMBLE

In [40]:
import pandas as pd 
import io 
import time
import wget
#wget.download("https://raw.githubusercontent.com/arghac14/UndErNsembled/master/Datasets/pima-diabetes.csv")
df=pd.read_csv("pima-diabetes (1).csv",header=None)

In [41]:
import numpy as np

In [42]:
df.shape

(769, 9)

In [43]:
#df.Outcome.value_counts()

In [44]:
df.isnull().sum()# no null values

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
dtype: int64

In [45]:
df.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')

In [46]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt

In [47]:
model=ExtraTreesClassifier()
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
model.fit(X,y)

ValueError: could not convert string to float: 'Pregnancies'

In [37]:
print((model.feature_importances_))

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [38]:
# plotting importances
feature_imp=pd.Series(model.feature_importances_,index=X.columns)
feature_imp.plot(kind='barh')
plt.show()

NotFittedError: This ExtraTreesClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [39]:
# modifying the dataset
cols=["Glucose","BMI","DiabetesPedigreeFunction","Age","Outcome"]
df_modified=df[cols]

KeyError: "None of [Index(['Glucose', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'], dtype='object')] are in the [columns]"

In [ ]:
df_modified.head()

In [ ]:
#dividing into train and test samples (120,30)
from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df_modified, test_size=0.20)

In [ ]:
df_train.head()

## bootstrap

In [57]:
percentage=0.5
k=50
bootstrapped_datasets = []
for i in range(k):from sklearn.model_selection import train_test_split
df_train,df_test=train_test_split(df_modified, test_size=0.20)
  bootstrapped_datasets.append(df_train.sample(frac = percentage))

In [58]:
len(bootstrapped_datasets)

50

In [59]:
#out of bag sampling
out_of_bag_datasets = []

for i in range(len(bootstrapped_datasets)):
  out_of_bag = []
  row = []
  dataset = bootstrapped_datasets[i] #corrrect
  #print(dataset)
  c = 0
  for index in range(len(df_train)):
      row = list(df_train.iloc[index])
      #print(dataset.values.tolist()) 
      if row not in dataset.values.tolist():
        c+=1
        out_of_bag.append(row)
  out_of_bag_datasets.append(out_of_bag)

In [60]:
out_bag_datasets = []
cols = df_train.columns

data = []
for dataset in out_of_bag_datasets:
  out_bag_datasets.append(pd.DataFrame(dataset, columns = cols))
out_bag_datasets[0].head()

,Glucose,BMI,DiabetesPedigreeFunction,Age,Outcome
0,131.0,33.1,0.160,28.0,0.0
1,80.0,26.2,0.313,41.0,0.0
2,102.0,25.1,0.078,21.0,0.0
3,0.0,27.7,0.299,21.0,0.0
4,111.0,26.2,0.343,23.0,0.0


## Model training with bootstraped samples

In [61]:
#Using Decision trees, SVM,KNN,NAIVE BAYYES,LOGISTIC REG
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm

In [1]:
def classify(dataset,out_of_bag):
  lr = LogisticRegression(max_iter=1000,multi_class="ovr")
  kn = KNeighborsClassifier(n_neighbors = 10)
  nb = GaussianNB()
  sv = svm.SVC()
  dt = DecisionTreeClassifier()
                                     
  classifiers = ['lr', 'kn', 'nb', 'svm', 'dt']
  models = [lr, kn, nb, sv, dt]    
  classifier_accuracy = []  ## Check index wise
  X = dataset.iloc[:,:-1] # upto target column
  y = dataset["Outcome"] # only target column
 # X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)
  y_pred = []
  X_test=out_of_bag.iloc[:,:-1]
  y_test=out_of_bag["Outcome"]
  #cols = X_train.columns
  #X2_test = pd.DataFrame(colon_df_test, columns= cols)
  #y2_test = colon_df_test[target] 
  for m in models:
    m.fit(X,y.astype("int"))
    y_pred = m.predict(X_test)
    classifier_accuracy.append(round(accuracy_score(y_test,y_pred)*100,6))
  
  best_accuracy = max(classifier_accuracy)
 # print(best_accuracy)
  best_model = models[classifier_accuracy.index(best_accuracy)]
  best_modelname=classifiers[classifier_accuracy.index(best_accuracy)]
 
  best_model.fit(X,y.astype(int))
  pred=best_model.predict(df_test.iloc[:,:-1])
  #print(classifier_accuracy)
  
  return pred,best_modelname

In [2]:
test_predictions=[]
model=[]
for i in range(0,50):
    cl1,ml1=classify(bootstrapped_datasets[i],out_bag_datasets[i])
    test_predictions.append(cl1)
    model.append(ml1)

NameError: name 'bootstrapped_datasets' is not defined

In [3]:
from scipy import stats as st


In [65]:
def majority_voting(preds):
    k=0
    l=[]
    k=[]
    for i in range(0,len(preds[0])):
        l=[]
        for j in range(0,len(preds)):
            l.append(preds[j][i])
        
        k.append(max(set(l), key=l.count))
    return k
    

In [66]:
finalpreds=majority_voting(test_predictions)

In [67]:
print(finalpreds)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1]


In [68]:
#accuracy check for m-bagging model
accuracy=accuracy_score(df_test["Outcome"],finalpreds)

In [69]:
print(accuracy)

0.7857142857142857


In [70]:
for each in set(model):
    k=model.count(each)
    print(each,":",k)


svm : 4
nb : 16
dt : 1
kn : 1
lr : 28


In [71]:
#0.2 test data: 0.811
#0.3 test data: 0.783
#0.4 test data: 0.791
#0.5 test data: 0.781

In [73]:
from sklearn.model_selection import KFold 
train=[]
test=[]
kfold=KFold(10,shuffle=True)

for x,y in kfold.split(df_modified):
    train.append(x)
    test.append(y)


In [74]:
 model=[]
accuracy_10Fold=[]
for i in range(0,10):
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    print(len(df_train))
    percentage=0.5
    k=50
    bootstrapped_datasets = []
    for i in range(k):
      bootstrapped_datasets.append(df_train.sample(frac = percentage))
    print(len(bootstrapped_datasets[0]))
    out_bag_datasets = []
    cols = df_train.columns
    out_of_bag_datasets = []

    for i in range(len(bootstrapped_datasets)):
      out_of_bag = []
      row = []
      dataset = bootstrapped_datasets[i] #corrrect
      #print(dataset)
      c = 0
      for index in range(len(df_train)):
          row = list(df_train.iloc[index])
          #print(dataset.values.tolist()) 
          if row not in dataset.values.tolist():
            c+=1
            out_of_bag.append(row)
      out_of_bag_datasets.append(out_of_bag)
    
    for dataset in out_of_bag_datasets:
      out_bag_datasets.append(pd.DataFrame(dataset, columns = cols))
    test_predictions=[]
   
    for i in range(0,50):
        cl1,ml1=classify(bootstrapped_datasets[i],out_bag_datasets[i])
        test_predictions.append(cl1)
        model.append(ml1)
    finalpreds=majority_voting(test_predictions)
    #print(len(test_predictions))
    accuracy=accuracy_score(df_test["Outcome"],finalpreds)
    accuracy_10Fold.append(accuracy)

        
    

691
346
691
346
691
346
691
346
691
346
691
346
691
346
691
346
692
346
692
346


In [75]:
print(np.mean(accuracy_10Fold))

0.7683185235816815


In [76]:
from sklearn.metrics import confusion_matrix as cm
cm(df_test["Outcome"],finalpreds)

array([[43,  5],
       [ 8, 20]], dtype=int64)

In [77]:
#0.7 10 fold 0.781

In [78]:
#0.5 10 fold 0.78

In [79]:
for each in set(model):
    k=model.count(each)
    print(each,":",k)

svm : 36
kn : 54
nb : 206
lr : 204


In [42]:
lr =DecisionTreeClassifier()
accuracy_10Fold=[]
for i in range(0,10): 
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    
    lr.fit(df_train.iloc[:,:-1],df_train["Outcome"])
    pred=lr.predict(df_test.iloc[:,:-1])
    accuracy=accuracy_score(df_test["Outcome"],pred)
    accuracy_10Fold.append((accuracy))

IndexError: list index out of range

In [43]:
print(np.mean(accuracy_10Fold))

nan


C:\Users\Abhishek\anaconda new\envs\gpu_env\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Abhishek\anaconda new\envs\gpu_env\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [38]:
lr =LogisticRegression()
accuracy_10Fold=[]
for i in range(0,10): 
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    
    lr.fit(df_train.iloc[:,:-1],df_train["Outcome"])
    pred=lr.predict(df_test.iloc[:,:-1])
    accuracy=accuracy_score(df_test["Outcome"],pred)
    accuracy_10Fold.append((accuracy))
print(np.mean(accuracy_10Fold))

C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)


0.7656869446343132


C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\linear_model\base.py:291: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  indices = (scores > 0).astype(np.int)
C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solv

In [44]:
lr =svm.SVC()
accuracy_10Fold=[]
for i in range(0,10): 
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    
    lr.fit(df_train.iloc[:,:-1],df_train["Outcome"])
    pred=lr.predict(df_test.iloc[:,:-1])
    accuracy=accuracy_score(df_test["Outcome"],pred)
    accuracy_10Fold.append((accuracy))
print(np.mean(accuracy_10Fold))

IndexError: list index out of range

In [40]:
lr =GaussianNB()
accuracy_10Fold=[]
for i in range(0,10): 
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    
    lr.fit(df_train.iloc[:,:-1],df_train["Outcome"])
    pred=lr.predict(df_test.iloc[:,:-1])
    accuracy=accuracy_score(df_test["Outcome"],pred)
    accuracy_10Fold.append((accuracy))
print(np.mean(accuracy_10Fold))

0.7721633629528367


In [41]:
lr =KNeighborsClassifier()
accuracy_10Fold=[]
for i in range(0,10): 
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    
    lr.fit(df_train.iloc[:,:-1],df_train["Outcome"])
    pred=lr.predict(df_test.iloc[:,:-1])
    accuracy=accuracy_score(df_test["Outcome"],pred)
    accuracy_10Fold.append((accuracy))
print(np.mean(accuracy_10Fold))

C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:908: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._y = np.empty(y.shape, dtype=np.int)
C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:908: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your

0.7227785372522215


In [42]:
dt = KNeighborsClassifier()
dt.fit(df_train.iloc[:,:-1],df_train["Outcome"])
pred=dt.predict(df_test.iloc[:,:-1])
accuracy_score(df_test["Outcome"],pred)

C:\Users\ABHISHEK PAUL\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:908: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self._y = np.empty(y.shape, dtype=np.int)


0.7631578947368421

# new way of bootstraping

In [43]:
percentage=0.25
t=4
k=50
bootstrapped_datasets = []
for i in range(k):
    p=pd.DataFrame(columns=df_train.columns)
    for i in range(t):
        l=df_train.sample(frac = percentage)
        p= pd.concat([p,l])
    bootstrapped_datasets.append(p)

AttributeError: type object 'object' has no attribute 'dtype'

In [ ]:
#out of bag sampling
out_of_bag_datasets = []

for i in range(len(bootstrapped_datasets)):
  out_of_bag = []
  row = []
  dataset = bootstrapped_datasets[i] #corrrect
  #print(dataset)
  c = 0
  for index in range(len(df_train)):
      row = list(df_train.iloc[index])
      #print(dataset.values.tolist()) 
      if row not in dataset.values.tolist():
        c+=1
        out_of_bag.append(row)
  out_of_bag_datasets.append(out_of_bag)

In [ ]:
out_bag_datasets = []
cols = df_train.columns

data = []
for dataset in out_of_bag_datasets:
  out_bag_datasets.append(pd.DataFrame(dataset, columns = cols))
len(out_bag_datasets[0])

In [ ]:
test_predictions=[]
model=[]
for i in range(0,50):
    cl1,ml1=classify(bootstrapped_datasets[i],out_bag_datasets[i]);
    test_predictions.append(cl1)
    model.append(ml1);

In [ ]:
# model frequency
for each in set(model):
    k=model.count(each)
    print(each,":",k)

In [ ]:
finalpreds=majority_voting(test_predictions)
accuracy=accuracy_score(df_test["Outcome"],finalpreds)
print(accuracy)

In [18]:
#new bootstrapping 10 Fold
model=[]
accuracy_10Fold=[]
for i in range(0,10):
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    print(len(df_train))
    percentage=0.7
    k=50
    percentage=0.25
    t=4
    k=50
    bootstrapped_datasets = []
    for i in range(k):
        p=pd.DataFrame(columns=df_train.columns)
        for i in range(t):
            l=df_train.sample(frac = percentage)
            p= pd.concat([p,l])
        bootstrapped_datasets.append(p)
    print(len(bootstrapped_datasets[0]))
    out_bag_datasets = []
    cols = df_train.columns
    out_of_bag_datasets = []

    for i in range(len(bootstrapped_datasets)):
      out_of_bag = []
      row = []
      dataset = bootstrapped_datasets[i] #corrrect
      #print(dataset)
      c = 0
      for index in range(len(df_train)):
          row = list(df_train.iloc[index])
          #print(dataset.values.tolist()) 
          if row not in dataset.values.tolist():
            c+=1
            out_of_bag.append(row)
      out_of_bag_datasets.append(out_of_bag)
    
    for dataset in out_of_bag_datasets:
      out_bag_datasets.append(pd.DataFrame(dataset, columns = cols))
    test_predictions=[]
   
    for i in range(0,50):
        cl1,ml1=classify(bootstrapped_datasets[i],out_bag_datasets[i])
        test_predictions.append(cl1)
        model.append(ml1)
    finalpreds=majority_voting(test_predictions)
    #print(len(test_predictions))
    accuracy=accuracy_score(df_test["Outcome"],finalpreds)
    accuracy_10Fold.append(accuracy)

        
    

NameError: name 'df_modified' is not defined

In [ ]:
print(np.mean(accuracy_10Fold))

# homogeneous bagging

In [ ]:
from sklearn.model_selection import KFold 
train=[]
test=[]
kfold=KFold(10,True)

for x,y in kfold.split(df_modified):
    train.append(x)
    test.append(y)

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
aclist=[]
for i in range(10):
    df_train=df_modified.iloc[train[i]]
    df_test=df_modified.iloc[test[i]]
    X=df_train.iloc[:,:-1]
    y=df_train["Outcome"]
    clf = BaggingClassifier(base_estimator=svm.SVC(),n_estimators=50, random_state=0).fit(X, y)
    ac=accuracy_score(df_test.Outcome,clf.predict(df_test.iloc[:,:-1]))
    aclist.append(ac)
print(np.mean(aclist));

dt=0.746
lr=0.769
knn=0.734
nb=0.771
svm=0.6236



